In [1]:
import os
import sys
import tempfile
from ast import literal_eval

import tensorflow as tf

import fastestimator as fe
from fastestimator.architecture.retinanet import RetinaNet, get_fpn_anchor_box, get_target
from fastestimator.dataset.mscoco import load_data
from fastestimator.op import NumpyOp, TensorOp
from fastestimator.op.numpyop import ImageReader, ResizeImageAndBbox, TypeConverter
from fastestimator.op.tensorop import Loss, ModelOp, Pad, Rescale
from fastestimator.trace import ModelSaver

In [2]:
train_csv, val_csv, path = load_data(path='/data/hsiming/dataset/')

In [3]:
class String2List(NumpyOp):
    # this thing converts '[1, 2, 3]' into np.array([1, 2, 3])
    def forward(self, data, state):
        data = map(literal_eval, data)
        return data


class GenerateTarget(NumpyOp):
    def __init__(self, inputs=None, outputs=None, mode=None):
        super().__init__(inputs=inputs, outputs=outputs, mode=mode)
        self.anchorbox, _ = get_fpn_anchor_box(input_shape=(512, 512, 3))

    def forward(self, data, state):
        obj_label, x1, y1, width, height = data
        cls_gt, x1_gt, y1_gt, w_gt, h_gt = get_target(self.anchorbox, obj_label, x1, y1, width, height)
        return cls_gt, x1_gt, y1_gt, w_gt, h_gt


class RetinaLoss(Loss):
    def focal_loss(self, cls_gt_example, cls_pred_example, alpha=0.25, gamma=2.0):
        # cls_gt_example shape: [A], cls_pred_example shape: [A, K]
        num_classes = cls_pred_example.shape[-1]
        # gather the objects and background, discard the rest
        anchor_obj_idx = tf.where(tf.greater_equal(cls_gt_example, 0))
        anchor_obj_bg_idx = tf.where(tf.greater_equal(cls_gt_example, -1))
        anchor_obj_count = tf.cast(tf.shape(anchor_obj_idx)[0], tf.float32)
        cls_gt_example = tf.one_hot(cls_gt_example, num_classes)
        cls_gt_example = tf.gather_nd(cls_gt_example, anchor_obj_bg_idx)
        cls_pred_example = tf.gather_nd(cls_pred_example, anchor_obj_bg_idx)
        cls_gt_example = tf.reshape(cls_gt_example, (-1, 1))
        cls_pred_example = tf.reshape(cls_pred_example, (-1, 1))
        # compute the focal weight on each selected anchor box
        alpha_factor = tf.ones_like(cls_gt_example) * alpha
        alpha_factor = tf.where(tf.equal(cls_gt_example, 1), alpha_factor, 1 - alpha_factor)
        focal_weight = tf.where(tf.equal(cls_gt_example, 1), 1 - cls_pred_example, cls_pred_example)
        focal_weight = alpha_factor * focal_weight**gamma / anchor_obj_count
        cls_loss = tf.losses.BinaryCrossentropy(reduction='sum')(cls_gt_example,
                                                                 cls_pred_example,
                                                                 sample_weight=focal_weight)
        return cls_loss, anchor_obj_idx

    def smooth_l1(self, loc_gt_example, loc_pred_example, anchor_obj_idx, beta=0.1):
        """Return smooth l1 loss for box regesssion.

        Args:
            loc_gt_example (Tensor): Tensor of shape (padded=252, 4).
            loc_pred_example (Tensor): Tensor of shape (num_anchors, 4).
            anchor_obj_idx (Tensor): Indices of selected anchor box.

        Returns:
            float: Smooth l1 loss.
        """
        loc_pred = tf.gather_nd(loc_pred_example, anchor_obj_idx)  #anchor_obj_count x 4
        anchor_obj_count = tf.shape(loc_pred)[0]
        loc_gt = loc_gt_example[:anchor_obj_count]  #anchor_obj_count x 4
        loc_gt = tf.reshape(loc_gt, (-1, 1))
        loc_pred = tf.reshape(loc_pred, (-1, 1))
        loc_diff = tf.abs(loc_gt - loc_pred)
        cond = tf.less(loc_diff, beta)
        smooth_l1_loss = tf.where(cond, 0.5 * loc_diff**2 / beta, loc_diff - 0.5 * beta)
        smooth_l1_loss = tf.reduce_sum(smooth_l1_loss) / tf.cast(anchor_obj_count, tf.float32)
        return smooth_l1_loss

    def forward(self, data, state):
        cls_gt, x1_gt, y1_gt, w_gt, h_gt, cls_pred, loc_pred = data
        local_batch_size = state["local_batch_size"]
        focal_loss = []
        l1_loss = []
        total_loss = []
        for idx in range(local_batch_size):
            cls_gt_example = cls_gt[idx]
            x1_gt_example = x1_gt[idx]
            y1_gt_example = y1_gt[idx]
            w_gt_example = w_gt[idx]
            h_gt_example = h_gt[idx]
            loc_gt_example = tf.transpose(tf.stack([x1_gt_example, y1_gt_example, w_gt_example, h_gt_example]))
            cls_pred_example = cls_pred[idx]
            loc_pred_example = loc_pred[idx]
            focal_loss_example, anchor_obj_idx = self.focal_loss(cls_gt_example, cls_pred_example)
            smooth_l1_loss_example = self.smooth_l1(loc_gt_example, loc_pred_example, anchor_obj_idx)
            focal_loss.append(focal_loss_example)
            l1_loss.append(smooth_l1_loss_example)
        focal_loss = tf.stack(focal_loss)
        l1_loss = tf.stack(l1_loss)
        total_loss = focal_loss + l1_loss

        return total_loss, focal_loss, l1_loss

In [4]:
class PredictBox(TensorOp):
    def __init__(self,
                 inputs=None,
                 outputs=None,
                 mode=None,
                 input_shape=(512, 512, 3),
                 select_top_k=1000,
                 nms_max_outputs=100):
        super().__init__(inputs=inputs, outputs=outputs, mode=mode)
        self.input_shape = input_shape
        self.select_top_k = tf.cast(select_top_k, dtype=tf.int32)
        self.nms_max_outputs = nms_max_outputs

        all_anchors, num_anchors_per_level = get_fpn_anchor_box(input_shape=input_shape)
        self.all_anchors = tf.convert_to_tensor(all_anchors)
        self.num_anchors_per_level = tf.convert_to_tensor(num_anchors_per_level, dtype=tf.int32)

    def index_to_bool(self, indices, length):
        updates = tf.ones_like(indices, dtype=tf.bool)
        shape = tf.expand_dims(length, 0)
        is_selected = tf.scatter_nd(tf.cast(tf.expand_dims(indices, axis=-1), dtype=tf.int32), updates, shape)
        return is_selected

    def forward(self, data, state):
        """
        # state['local_batch_size']
        # cls_pred.shape = (8, 49104, 90)
        """

        pred = []
        gt = []

        # extract max score and its class label
        #cls_pred, deltas, label_gt, x1_gt, y1_gt, w_gt, h_gt = data
        cls_pred, deltas = data
        #label_gt = tf.convert_to_tensor(label_gt)
        #x1_gt = tf.convert_to_tensor(x1_gt)
        #y1_gt = tf.convert_to_tensor(y1_gt)
        #w_gt = tf.convert_to_tensor(w_gt)
        #h_gt = tf.convert_to_tensor(h_gt)

        labels = tf.cast(tf.argmax(cls_pred, axis=2), dtype=tf.int32)
        scores = tf.reduce_max(cls_pred, axis=2)

        # iterate over image
        for i in range(state['local_batch_size']):
            labels_per_image = labels[i]
            scores_per_image = scores[i]
            deltas_per_image = deltas[i]
            
            #keep_gt = tf.greater(label_gt[i], 0)
            #label_gt_per_image = label_gt[i][keep_gt]
            #x1_gt_per_image = x1_gt[i][keep_gt]
            #y1_gt_per_image = y1_gt[i][keep_gt]
            #w_gt_per_image = w_gt[i][keep_gt]
            #h_gt_per_image = h_gt[i][keep_gt]

            selected_deltas_per_image = tf.constant([], shape=(0, 4))
            selected_labels_per_image = tf.constant([], dtype=tf.int32)
            selected_scores_per_image = tf.constant([])
            selected_anchor_indices_per_image = tf.constant([], dtype=tf.int32)

            end_index = 0
            # iterate over each pyramid level
            for j in range(self.num_anchors_per_level.shape[0]):
                start_index = end_index
                end_index += self.num_anchors_per_level[j]
                anchor_indices = tf.range(start_index, end_index, dtype=tf.int32)

                level_scores = scores_per_image[start_index:end_index]
                level_deltas = deltas_per_image[start_index:end_index]
                level_labels = labels_per_image[start_index:end_index]

                # select top k
                if self.num_anchors_per_level[j] >= self.select_top_k:
                    # won't work without the tf.minimum
                    top_k = tf.math.top_k(level_scores, tf.minimum(self.num_anchors_per_level[j], self.select_top_k))
                    top_k_scores = top_k.values
                    top_k_indices = tf.add(top_k.indices, [start_index])
                else:
                    top_k_scores = level_scores
                    top_k_indices = anchor_indices

                # filter out low score
                is_high_score = tf.greater(top_k_scores, 0.05)
                selected_indices = tf.boolean_mask(top_k_indices, is_high_score)
                is_selected = self.index_to_bool(tf.subtract(selected_indices, [start_index]),
                                                 self.num_anchors_per_level[j])

                # combine all pyramid levels
                selected_deltas_per_image = tf.concat(
                    [selected_deltas_per_image, tf.boolean_mask(level_deltas, is_selected)], axis=0)
                selected_scores_per_image = tf.concat(
                    [selected_scores_per_image, tf.boolean_mask(level_scores, is_selected)], axis=0)
                selected_labels_per_image = tf.concat(
                    [selected_labels_per_image, tf.boolean_mask(level_labels, is_selected)], axis=0)
                selected_anchor_indices_per_image = tf.concat(
                    [selected_anchor_indices_per_image, tf.boolean_mask(anchor_indices, is_selected)], axis=0)

            # delta -> (x1, y1, w, h)
            anchor_mask = self.index_to_bool(selected_anchor_indices_per_image, self.all_anchors.shape[0])
            x1 = (selected_deltas_per_image[:, 0] * tf.boolean_mask(
                self.all_anchors, anchor_mask)[:, 2]) + tf.boolean_mask(self.all_anchors, anchor_mask)[:, 0]
            y1 = (selected_deltas_per_image[:, 1] * tf.boolean_mask(
                self.all_anchors, anchor_mask)[:, 3]) + tf.boolean_mask(self.all_anchors, anchor_mask)[:, 1]
            w = tf.math.exp(selected_deltas_per_image[:, 2]) * tf.boolean_mask(self.all_anchors, anchor_mask)[:, 2]
            h = tf.math.exp(selected_deltas_per_image[:, 3]) * tf.boolean_mask(self.all_anchors, anchor_mask)[:, 3]
            x2 = x1 + w
            y2 = y1 + h

            # nms
            boxes_per_image = tf.stack([y1, x1, y2, x2], axis=1)
            nms_indices = tf.image.non_max_suppression(boxes_per_image, selected_scores_per_image, self.nms_max_outputs)

            nms_boxes = tf.gather(boxes_per_image, nms_indices)
            final_scores = tf.gather(selected_scores_per_image, nms_indices)
            final_labels = tf.gather(selected_labels_per_image, nms_indices)

            x1 = tf.clip_by_value(nms_boxes[:, 1], clip_value_min=0, clip_value_max=self.input_shape[1])
            y1 = tf.clip_by_value(nms_boxes[:, 0], clip_value_min=0, clip_value_max=self.input_shape[0])
            w = tf.clip_by_value(nms_boxes[:, 3], clip_value_min=0, clip_value_max=self.input_shape[1]) - x1
            h = tf.clip_by_value(nms_boxes[:, 2], clip_value_min=0, clip_value_max=self.input_shape[0]) - y1

            final_boxes = tf.stack([x1, y1, w, h], axis=1)

            # combine image results into batch
            image_results = tf.concat([
                tf.pad(final_boxes, [[0, 0], [1, 0]], constant_values=i),
                tf.cast(tf.expand_dims(final_labels, axis=1), dtype=tf.float32),
                tf.expand_dims(final_scores, axis=1)
            ],
                                      axis=1)

            #image_gt = tf.transpose(
            #    tf.concat([
            #        tf.stack([i * tf.ones_like(x1_gt_per_image), x1_gt_per_image]),
            #        tf.expand_dims(y1_gt_per_image, axis=0),
            #        tf.expand_dims(w_gt_per_image, axis=0),
            #        tf.expand_dims(h_gt_per_image, axis=0),
            #        tf.expand_dims(label_gt_per_image, axis=0)
            #    ],
            #              axis=0))
            pred.append(image_results)
            #gt.append(image_gt)
            tf.print(final_scores)
        return tf.concat(pred, axis=0)  #, tf.concat(gt, axis=0)

In [5]:
model_dir = '/data/hsiming/mscoco_model/'
writer = fe.RecordWriter(
    save_dir=os.path.join(path, "retinanet_coco_small"),
    train_data='/data/hsiming/dataset/MSCOCO2017/train_object_small.csv',
    validation_data='/data/hsiming/dataset/MSCOCO2017/train_object_small.csv',
    ops=[
        ImageReader(inputs="image", parent_path=path, outputs="image"),
        String2List(inputs=["x1", "y1", "width", "height", "obj_label"],
                    outputs=["x1", "y1", "width", "height", "obj_label"]),
        ResizeImageAndBbox(target_size=(512, 512),
                           keep_ratio=True,
                           inputs=["image", "x1", "y1", "width", "height"],
                           outputs=["image", "x1", "y1", "width", "height"]),
        GenerateTarget(inputs=("obj_label", "x1", "y1", "width", "height"),
                       outputs=("cls_gt", "x1_gt", "y1_gt", "w_gt", "h_gt")),
        TypeConverter(target_type='int32', inputs=["id", "cls_gt"], outputs=["id", "cls_gt"]),
        TypeConverter(target_type='float32',
                      inputs=["x1_gt", "y1_gt", "w_gt", "h_gt"],
                      outputs=["x1_gt", "y1_gt", "w_gt", "h_gt"])
    ],
    compression="GZIP",
    write_feature=["image", "id", "cls_gt", "x1_gt", "y1_gt", "w_gt", "h_gt"])

# prepare pipeline
pipeline = fe.Pipeline(
    batch_size=8,
    padded_batch=True,
    data=writer,
    ops=[
        Rescale(inputs="image", outputs="image"),
        Pad(padded_shape=[190], inputs=["x1_gt", "y1_gt", "w_gt", "h_gt"], outputs=["x1_gt", "y1_gt", "w_gt", "h_gt"])
    ])

# prepare network
model = fe.build(model_def=lambda: RetinaNet(input_shape=(512, 512, 3), num_classes=90),
                 model_name="retinanet",
                 optimizer=tf.optimizers.Adam(learning_rate=0.0002),
                 loss_name="total_loss")
network = fe.Network(ops=[
    ModelOp(inputs="image", model=model, outputs=["cls_pred", "loc_pred"]),
    PredictBox(inputs=["cls_pred", "loc_pred"],
               outputs=("pred"),
               mode="eval"),
    RetinaLoss(inputs=("cls_gt", "x1_gt", "y1_gt", "w_gt", "h_gt", "cls_pred", "loc_pred"),
               outputs=("total_loss", "focal_loss", "l1_loss"))
])

# prepare estimator
estimator = fe.Estimator(
    network=network,
    pipeline=pipeline,
    epochs=200,
    #steps_per_epoch=2,
    log_steps=1,
    #validation_steps=2,
    traces=ModelSaver(model_name="retinanet", save_dir=model_dir, save_best=True))

In [6]:
estimator.fit()

    ______           __  ______     __  _                 __            
   / ____/___ ______/ /_/ ____/____/ /_(_)___ ___  ____ _/ /_____  _____
  / /_  / __ `/ ___/ __/ __/ / ___/ __/ / __ `__ \/ __ `/ __/ __ \/ ___/
 / __/ / /_/ (__  ) /_/ /___(__  ) /_/ / / / / / / /_/ / /_/ /_/ / /    
/_/    \__,_/____/\__/_____/____/\__/_/_/ /_/ /_/\__,_/\__/\____/_/     
                                                                        

FastEstimator: Reading non-empty directory: /data/hsiming/dataset/MSCOCO2017/retinanet_coco_small
FastEstimator: Found 50 examples for train in /data/hsiming/dataset/MSCOCO2017/retinanet_coco_small/train_summary0.json
FastEstimator: Found 50 examples for eval in /data/hsiming/dataset/MSCOCO2017/retinanet_coco_small/eval_summary0.json
[]
[]
[]
[]
[]
[]
[]
[]
FastEstimator-Start: step: 0; total_train_steps: 1200; retinanet_lr: 0.0002; 
FastEstimator-Train: step: 0; focal_loss: 2.1004574; l1_loss: 0.9493042; total_loss: 3.0497618; 
FastEstimator-Train: step:

FastEstimator-Train: step: 40; focal_loss: 1.0125945; l1_loss: 0.6951562; total_loss: 1.7077508; examples/sec: 31.8; progress: 3.3%; 
FastEstimator-Train: step: 41; focal_loss: 1.6896902; l1_loss: 0.5435516; total_loss: 2.2332416; examples/sec: 32.4; progress: 3.4%; 
[0.999990344 0.999976516 0.999874592 ... 0.0867998898 0.0821536779 0.074852705]
[0.999946356 0.999582052 0.998945355 ... 0.0637586117 0.0589213073 0.0526533127]
[0.998245716 0.997383 0.99127388 ... 0.0639478 0.0566522777 0.0512379706]
[0.999984 0.999952912 0.999782085 ... 0.0840135217 0.0801663399 0.0724467933]
[0.999999762 0.999999285 0.999997079 ... 0.239436626 0.236735612 0.173248261]
[0.99999 0.999966145 0.999850214 ... 0.10776028 0.0966772139 0.0939851105]
[0.998591065 0.994471669 0.987825155 ... 0.0586817861 0.0541267693 0.0511571467]
[0.998968244 0.997483611 0.992986321 ... 0.0565140843 0.0527821779 0.051310569]
[0.999959648 0.999933124 0.999650717 ... 0.087109983 0.0790414512 0.0789936781]
[0.998859763 0.996772289 

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
FastEstimator-Eval: step: 66; epoch: 10; focal_loss: 1.7068847; l1_loss: 0.7050376; total_loss: 2.4119222; 
FastEstimator-Train: step: 66; focal_loss: 0.9251978; l1_loss: 0.4014777; total_loss: 1.3266757; examples/sec: 32.2; progress: 5.5%; 
FastEstimator-Train: step: 67; focal_loss: 0.901158; l1_loss: 0.4899395; total_loss: 1.3910975; examples/sec: 32.3; progress: 5.6%; 
FastEstimator-Train: step: 68; focal_loss: 0.9769981; l1_loss: 0.5195095; total_loss: 1.4965076; examples/sec: 32.0; progress: 5.7%; 
FastEstimator-Train: step: 69; focal_loss: 0.951048; l1_loss: 0.5391904; total_loss: 1.4902385; examples/sec: 31.9; progress: 5.8%; 
FastEstimator-Train: step: 70; focal_loss: 1.1865665; l1_loss: 0.4966197; total_loss: 1.6831862; examples/sec: 31.5; progress: 5.8%; 
FastEstimator-Train: step: 71; focal_loss: 0.855185; l1_loss: 0.3893334; total_loss: 1.2445185; examples/sec: 31.9; progress: 5.9%; 
[]
[]
[]
[]
[]
[

FastEstimator-Train: step: 109; focal_loss: 0.8596258; l1_loss: 0.505895; total_loss: 1.3655207; examples/sec: 32.1; progress: 9.1%; 
FastEstimator-Train: step: 110; focal_loss: 0.8301064; l1_loss: 0.3836161; total_loss: 1.2137225; examples/sec: 32.3; progress: 9.2%; 
FastEstimator-Train: step: 111; focal_loss: 0.712482; l1_loss: 0.3897237; total_loss: 1.1022058; examples/sec: 32.2; progress: 9.2%; 
FastEstimator-Train: step: 112; focal_loss: 0.6941641; l1_loss: 0.3090229; total_loss: 1.003187; examples/sec: 31.6; progress: 9.3%; 
FastEstimator-Train: step: 113; focal_loss: 0.8082272; l1_loss: 0.3516878; total_loss: 1.1599151; examples/sec: 32.0; progress: 9.4%; 
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
FastEstimator-Eval: step: 114; epoch: 18; focal_loss: 1.8645892; l1_loss: 0.6885447; total_loss: 2.5531337; 
FastEstimator-Train: step: 114; focal_loss: 0.7150674; l1_loss: 0.4032079; 

[0.0732933879 0.0667878091]
[0.0693983734 0.0635132492]
[0.0794959664 0.0718862116]
[0.0725059509 0.0661008954]
[0.0731957257 0.0666798949]
[0.0712218881 0.0649889708]
[0.0702788234 0.0643031299]
[0.0750010908 0.068380028]
[0.0754281878 0.0680479705]
[0.0702354908 0.0642284155]
[0.0698229074 0.063891083]
[0.0718272 0.0653479099]
[0.0712842941 0.0650559068]
[0.0756658316 0.0684907138]
[0.0703611374 0.0643287]
[0.0704133511 0.0643597245]
[0.0729327202 0.066041559]
[0.0759119093 0.0685941]
[0.0714818239 0.0651920736]
[0.0743719637 0.066973865]
[0.0703962147 0.064342171]
[0.0713973641 0.0652080178]
[0.0752164721 0.0685777068]
[0.0708459616 0.0647032]
[0.0751808584 0.0681744814]
FastEstimator-Eval: step: 144; epoch: 23; focal_loss: 1.6281838; l1_loss: 0.7138706; total_loss: 2.3420544; 
FastEstimator-Train: step: 144; focal_loss: 0.6102322; l1_loss: 0.2369818; total_loss: 0.847214; examples/sec: 32.2; progress: 12.0%; 
FastEstimator-Train: step: 145; focal_loss: 0.7052793; l1_loss: 0.2983068

...

[0.983158946 0.972241163 0.939984262 0.939568698 0.909360349 0.775012493]
[0.958786428]
[0.997584343 0.885712445 0.16134271]
[0.994644642 0.993539274 0.984738588 ... 0.0565697253 0.0514858961 0.0504267514]
[0.996638536 0.990379214 0.989850879 ... 0.056817472 0.0506723523 0.0505039692]
[0.973508894 0.969504416 0.963210404 ... 0.0803506076 0.0566682518 0.0522883832]
[0.97749877 0.966279805 0.954695463 ... 0.931790352 0.109916657 0.0628148913]
[0.965817451]
[0.975215733 0.961176515 0.917844892]
[0.999911427 0.999411285 0.998465419 ... 0.0634129345 0.0539101362 0.0503132939]
[0.994571686 0.985456705 0.978732765 ... 0.0549059212 0.0532291532 0.0504509509]
[0.989554286 0.986525297 0.968378782]
[0.996119559 0.99494791 0.994476557 ... 0.0552400649 0.0537348688 0.0524884462]
[0.971186101 0.940359354 0.923476458]
[0.989211559 0.958190143]
[0.994078279 0.993657827 0.993646801 0.990189552 0.956918359 0.874670386]
[0.998555422 0.996017277 0.993659377 ... 0.0538450778 0.0519611835 0.050337404]
[0.99

[0.98393774 0.981217742 0.978955925 ... 0.0700246692 0.0515356064 0.0503374934]
[0.977268279 0.972717166 0.968426585 ... 0.936736822 0.181967407 0.0570483208]
[0.973815501]
[0.981061935 0.973714828 0.94465661]
[0.999934196 0.999610662 0.998555183 ... 0.0677610338 0.0584693849 0.0504846871]
[0.995257735 0.991573334 0.983921707 ... 0.052028358 0.0509494245 0.0500200391]
[0.991757035 0.982370555 0.966526866]
[0.996196389 0.995723605 0.994330287 ... 0.0523805618 0.0521193743 0.0500293672]
[0.975750446 0.96510309 0.937152386 0.0534216166]
[0.987708 0.96109736]
[0.996453047 0.996215105 0.994590759 0.994522 0.97932744 0.915057421]
[0.998929739 0.998684883 0.996688545 ... 0.0535434186 0.052767247 0.0517692566]
[0.997540951 0.983913898 0.974987388 ... 0.0559244454 0.0557602048 0.0518254042]
[0.997828722 0.993316412 0.985303223 ... 0.0565785766 0.0547170043 0.0518681109]
[0.99481976 0.982507348 0.95204854]
[0.994388342 0.990576148 0.98789537 ... 0.0551533103 0.0551151037 0.0506641865]
[0.9969357

[0.96345377]
[0.983006954 0.977073848 0.94418788]
[0.999937296 0.999645472 0.998596787 ... 0.0625954866 0.05954808 0.0548291802]
[0.995919228 0.991513491 0.982761145 ... 0.0530097187 0.0510219634 0.0504653752]
[0.992566347 0.984474361 0.963393867]
[0.99559617 0.994524 0.994067073 ... 0.0529994071 0.0525803566 0.0503200889]
[0.983132064 0.970396 0.949117541 0.0545772314]
[0.98913157 0.962095499]
[0.996260226 0.995428205 0.993688 ... 0.976883948 0.913204193 0.20473218]
[0.998783529 0.998165 0.996455431 ... 0.0528102219 0.0520303547 0.0517666936]
[0.998225927 0.987273574 0.980788 ... 0.0552553833 0.0528133214 0.050604254]
[0.997650385 0.992910624 0.982821822 ... 0.0692884 0.0595223308 0.0502953529]
[0.99400425 0.982848883 0.949568629]
[0.994124293 0.989774346 0.987185 ... 0.0584331155 0.0579452813 0.050322175]
[0.997216821 0.989451349 0.984675288 ... 0.0531401038 0.0509673953 0.050344497]
[0.996676803 0.984623671]
[0.99909544 0.99688518 0.996670663 ... 0.0570357144 0.0564675927 0.05477574

[0.982882 0.965803146 0.937761188]
[0.990714192 0.962858617]
[0.995649219 0.995360315 0.994889 0.993571758 0.966434121 0.929196835]
[0.999231 0.997624636 0.996624649 ... 0.057516396 0.0531213 0.0506055057]
[0.998515 0.989513636 0.982176661 ... 0.0542086065 0.0510974526 0.0503168106]
[0.997651041 0.99350369 0.984591722 ... 0.0620988 0.0539366901 0.052472353]
[0.99292171 0.984896779 0.941773295]
[0.995464802 0.990967274 0.990533 ... 0.0635235 0.0601339042 0.0532009304]
[0.997146606 0.990137577 0.985755444 ... 0.0553256869 0.0514213741 0.0506465435]
[0.997453332 0.986898303]
FastEstimator-ModelSaver: Saving model to /data/hsiming/mscoco_model/retinanet_best_total_loss.h5
FastEstimator-Eval: step: 1188; epoch: 197; focal_loss: 0.0001074; l1_loss: 0.0072549; total_loss: 0.0073624; 
FastEstimator-Train: step: 1188; focal_loss: 0.0001246; l1_loss: 0.0138892; total_loss: 0.0140137; examples/sec: 31.7; progress: 99.0%; 
FastEstimator-Train: step: 1189; focal_loss: 0.0001591; l1_loss: 0.0105975;

[0.996474743 0.995357275 0.994942784 0.994439 0.964081526 0.937477291]
[0.999315 0.998096 0.99711442 ... 0.0580728054 0.0549633205 0.0507517457]
[0.998321 0.987276316 0.97950381 ... 0.0716424 0.0667195618 0.0579317212]
[0.997702 0.993004084 0.984800816 ... 0.0601166189 0.0516582429 0.051309824]
FastEstimator-Eval: step: 1200; epoch: 199; focal_loss: 9.52e-05; l1_loss: 0.0075625; total_loss: 0.0076577; 
FastEstimator-Finish: step: 1200; total_time: 508.51 sec; retinanet_lr: 0.0002; 
